In [2]:
import numpy as np  # sometimes needed to avoid mkl-service error
import sys
import os
import argparse
import logging
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, WandbLogger
from pytorch_lightning.plugins import DDPPlugin
from pytorch_lightning.utilities import rank_zero_only
import torch
from torchmdnet.module import LNNP
from torchmdnet import datasets, priors, models
from torchmdnet.data import DataModule
from torchmdnet.models import output_modules
from torchmdnet.models.utils import rbf_class_mapping, act_class_mapping
from torchmdnet.utils import LoadFromFile, LoadFromCheckpoint, save_argparse, number
from pathlib import Path
import wandb
import json

In [6]:

with open('commandline_args.txt', 'r') as f:
    args = json.load(f)

In [10]:
data = DataModule(args)
data.prepare_data()
data.setup("fit")


c:\Users\aminr\.conda\envs\pvd\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train 400, val 50, test 41


computing mean and std: 100%|██████████| 4/4 [00:00<00:00, 10.54it/s]


In [13]:
prior = None
if args["prior_model"]:
    assert hasattr(priors, args.prior_model), (
        f"Unknown prior model {args['prior_model']}. "
        f"Available models are {', '.join(priors.__all__)}"
    )
    # initialize the prior model
    prior = getattr(priors, args.prior_model)(dataset=data.dataset)
    args.prior_args = prior.get_init_args()

In [14]:
model = LNNP(args, prior_model=prior, mean=data.mean, std=data.std)

In [19]:
print(model)

LNNP(
  (model): TorchMD_Net(
    (representation_model): TorchMD_ET(hidden_channels=256, num_layers=8, num_rbf=64, rbf_type=expnorm, trainable_rbf=False, activation=silu, attn_activation=silu, neighbor_embedding=NeighborEmbeddingJittable_dd7ef0(
      (embedding): Embedding(100, 256)
      (distance_proj): Linear(in_features=64, out_features=256, bias=True)
      (combine): Linear(in_features=512, out_features=256, bias=True)
      (cutoff): CosineCutoff()
    ), num_heads=8, distance_influence=both, cutoff_lower=0.0, cutoff_upper=5.0)
    (output_model): EquivariantScalar(
      (output_network): ModuleList(
        (0): GatedEquivariantBlock(
          (vec1_proj): Linear(in_features=256, out_features=256, bias=False)
          (vec2_proj): Linear(in_features=256, out_features=128, bias=False)
          (update_net): Sequential(
            (0): Linear(in_features=512, out_features=256, bias=True)
            (1): SiLU()
            (2): Linear(in_features=256, out_features=256, bia